In [0]:
import os
import numpy as np
import pandas as pd
import keras
from keras import models
from keras import layers

In [0]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import make_scorer, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')

In [0]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
#from sklearn.utils.fixes import signature
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [0]:
import tensorflow as tf
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils, to_categorical
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import reuters
from tensorflow.keras.models import load_model

In [0]:
from keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000)

In [29]:
print("train_data ", train_data.shape)
print("train_labels ", train_labels.shape)

print("test_data ", test_data.shape)
print("test_labels ", test_labels.shape)

train_data  (8982,)
train_labels  (8982,)
test_data  (2246,)
test_labels  (2246,)


In [30]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in
train_data[0]])

print(decoded_newswire)
print(train_labels[0])

? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3
3


In [0]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [32]:
#백터화 및 정규화
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

print("x_train ", x_train.shape)
print("x_test ", x_test.shape)

x_train  (8982, 10000)
x_test  (2246, 10000)


In [33]:
#원핫인코딩
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

print("one_hot_train_labels ", one_hot_train_labels.shape)
print("one_hot_test_labels ", one_hot_test_labels.shape)

one_hot_train_labels  (8982, 46)
one_hot_test_labels  (2246, 46)


In [34]:
#validation setting

x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

print("x_val ", x_val.shape)
print("y_val ", y_val.shape)

print("partial_x_train ", partial_x_train.shape)
print("partial_y_train ", partial_y_train.shape)

x_val  (1000, 10000)
y_val  (1000, 46)
partial_x_train  (7982, 10000)
partial_y_train  (7982, 46)


In [35]:
model = models.Sequential()
model.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(46, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               2560256   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 46)                11822     
Total params: 2,637,870
Trainable params: 2,637,870
Non-trainable params: 0
_________________________________________________________________


In [36]:
NumEpochs = 10
BatchSize = 512

model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(partial_x_train, partial_y_train, epochs=NumEpochs, batch_size=BatchSize, validation_data=(x_val, y_val))

results = model.evaluate(x_val, y_val)
print("_"*100)
print("Test Loss and Accuracy")
print("results ", results)

history_dict = history.history
history_dict.keys()

Train on 7982 samples, validate on 1000 samples
Epoch 1/10
7982/7982 [==============================] - 3s 337us/step - loss: 15.7152 - accuracy: 0.4514 - val_loss: 6.1696 - val_accuracy: 0.5880
Epoch 2/10
7982/7982 [==============================] - 2s 310us/step - loss: 5.7336 - accuracy: 0.5679 - val_loss: 5.2019 - val_accuracy: 0.6110
Epoch 3/10
7982/7982 [==============================] - 2s 308us/step - loss: 4.9772 - accuracy: 0.5980 - val_loss: 4.5942 - val_accuracy: 0.6260
Epoch 4/10
7982/7982 [==============================] - 2s 310us/step - loss: 4.4339 - accuracy: 0.6234 - val_loss: 4.1233 - val_accuracy: 0.6630
Epoch 5/10
7982/7982 [==============================] - 2s 307us/step - loss: 4.0224 - accuracy: 0.6448 - val_loss: 3.7409 - val_accuracy: 0.6750
Epoch 6/10
7982/7982 [==============================] - 2s 311us/step - loss: 3.7235 - accuracy: 0.6523 - val_loss: 3.5154 - val_accuracy: 0.6780
Epoch 7/10
7982/7982 [==============================] - 2s 305us/step - los

dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy'])

In [37]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000, )))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                640064    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 46)                2990      
Total params: 647,214
Trainable params: 647,214
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [0]:
#검증 데이터셋
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


In [45]:
#모델 학습
history = model.fit(
    partial_x_train, 
    partial_y_train,
    epochs=10,
    batch_size=512,
    validation_data=(x_val, y_val)
)

Epoch 1/10
16/16 [==============================] - 1s 41ms/step - loss: 0.0848 - accuracy: 0.9619 - val_loss: 1.0676 - val_accuracy: 0.8180
Epoch 2/10
16/16 [==============================] - 1s 42ms/step - loss: 0.0909 - accuracy: 0.9614 - val_loss: 1.0819 - val_accuracy: 0.8080
Epoch 3/10
16/16 [==============================] - 1s 40ms/step - loss: 0.0873 - accuracy: 0.9614 - val_loss: 1.0882 - val_accuracy: 0.8150
Epoch 4/10
16/16 [==============================] - 1s 42ms/step - loss: 0.0835 - accuracy: 0.9620 - val_loss: 1.1278 - val_accuracy: 0.8100
Epoch 5/10
16/16 [==============================] - 1s 41ms/step - loss: 0.0839 - accuracy: 0.9628 - val_loss: 1.0958 - val_accuracy: 0.8080
Epoch 6/10
16/16 [==============================] - 1s 41ms/step - loss: 0.0788 - accuracy: 0.9623 - val_loss: 1.1133 - val_accuracy: 0.8060
Epoch 7/10
16/16 [==============================] - 1s 41ms/step - loss: 0.0819 - accuracy: 0.9632 - val_loss: 1.0876 - val_accuracy: 0.8090
Epoch 8/10
16

In [0]:
#데이터 레이블 인코딩 변경
y_train = np.array(train_labels)
y_test = np.array(test_labels)

x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = y_train[:1000]
partial_y_train = y_train[1000:]

In [47]:
#모델 작성
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000, )))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                640064    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 46)                2990      
Total params: 647,214
Trainable params: 647,214
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)

In [49]:
history = model.fit(
    partial_x_train, 
    partial_y_train,
    epochs=10,
    batch_size=512,
    validation_data=(x_val, y_val)
)

Epoch 1/10
16/16 [==============================] - 1s 45ms/step - loss: 3.2984 - acc: 0.3188 - val_loss: 2.6195 - val_acc: 0.3630
Epoch 2/10
16/16 [==============================] - 1s 40ms/step - loss: 2.0683 - acc: 0.5480 - val_loss: 1.6871 - val_acc: 0.6340
Epoch 3/10
16/16 [==============================] - 1s 64ms/step - loss: 1.3894 - acc: 0.7003 - val_loss: 1.2945 - val_acc: 0.7000
Epoch 4/10
16/16 [==============================] - 1s 68ms/step - loss: 1.0593 - acc: 0.7657 - val_loss: 1.1253 - val_acc: 0.7610
Epoch 5/10
16/16 [==============================] - 1s 68ms/step - loss: 0.8358 - acc: 0.8250 - val_loss: 1.0233 - val_acc: 0.7830
Epoch 6/10
16/16 [==============================] - 1s 68ms/step - loss: 0.6585 - acc: 0.8623 - val_loss: 0.9483 - val_acc: 0.8030
Epoch 7/10
16/16 [==============================] - 1s 68ms/step - loss: 0.5114 - acc: 0.8935 - val_loss: 0.9038 - val_acc: 0.8110
Epoch 8/10
16/16 [==============================] - 1s 65ms/step - loss: 0.4010 - a